In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from PIL import Image
# from torchinfo import summary
import torch
import os
import warnings
warnings.filterwarnings("ignore")
from typing import Tuple

from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

from datasets import load_dataset
import pandas as pd

torch.cuda.empty_cache()


In [2]:
print('Number CUDA Devices:', torch.cuda.device_count())
print ('Current cuda device: ', torch.cuda.current_device(), ' **May not correspond to nvidia-smi ID above, check visibility parameter')

Number CUDA Devices: 1
Current cuda device:  0  **May not correspond to nvidia-smi ID above, check visibility parameter


In [3]:
# dataset_folder_name = '../mnt/local/data/kalexu97/large_dataset'
# dataset_folder_name = '../mnt/local/data/kalexu97/short_dataset'
dataset_folder_name = '../mnt/local/data/kalexu97/large_balanced_dataset'
dataset = load_dataset("imagefolder", data_dir=dataset_folder_name)
train_dataset, test_dataset = dataset['train'], dataset['test']


Resolving data files:   0%|          | 0/3000 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/540 [00:00<?, ?it/s]

In [5]:
dataset_folder_name = '../mnt/local/data/kalexu97/large_dataset'

def load_dataset_path2images(dataset_folder_name):
    train_test_folders = os.listdir(dataset_folder_name)
    datasets = {}
    for trts_split in train_test_folders:
        class_folders = os.listdir(dataset_folder_name+'/'+trts_split)
        labels = []
        pathes = []
        for class_folder in class_folders:
            image_names = os.listdir(dataset_folder_name+'/'+trts_split+'/'+class_folder)
            image_names = [dataset_folder_name+'/'+trts_split+'/'+class_folder+'/'+x for x in image_names]
            class_labels = [int(class_folder)]*len(image_names)
            labels.extend(class_labels)
            pathes.extend(image_names)
        local_dataset = {'image_path': pathes, 'label':labels}
        datasets[trts_split] = pd.DataFrame.from_dict(local_dataset)

    return datasets

dataset = load_dataset_path2images(dataset_folder_name)

train_dataset = dataset['train']
test_dataset = dataset['test']



In [2]:
from sklearn.model_selection import train_test_split

labelsTable = pd.read_csv('../mnt/local/data/kalexu97/trainLabels.csv')
root_dir = '../mnt/local/data/kalexu97/train'
labelsTable['image_path'] = labelsTable['image'].apply(lambda x: os.path.join(root_dir, x+'.jpeg'))
labelsTable['label'] = labelsTable['level']

labelsTable = labelsTable.drop(columns=['image', 'level'], axis=1)

train_dataset, test_dataset = train_test_split(labelsTable, test_size=0.2)

In [4]:
dataset = load_dataset_path2images('../mnt/local/data/kalexu97/large_balanced_dataset')
train_dataset = dataset['train']
test_dataset = dataset['test']

In [6]:
# img = Image.open('../mnt/local/data/kalexu97/large_dataset/train/0/35062_left.jpeg')
# w, h = img.size
# # transform for square crop
# transform = T.CenterCrop(2000)
# img = transform(img)
# print(w, h)
# img

In [3]:

# oversampling just repeating minority class items 
# enought times to be equal to major dataset in size

##############################################################################################

# max_size = train_dataset['label'].value_counts().max()
# lst = [train_dataset]
def resample(_dataset, ratio = 2):
    min_size = _dataset['label'].value_counts().min()
    lst = []
    
    for class_index, group in _dataset.groupby('label'):
        # lst.append(group.sample(max_size-len(group), replace=True))
        if class_index == 0:
            lst.append(group.sample(min_size*ratio, replace=False))
        else:
            if len(group) > min_size*ratio:
                lst.append(group.sample(min_size*ratio, replace=False))
            else:
                lst.append(group)
                lst.append(group.sample(min_size*ratio-len(group), replace=True))
        
    _dataset = pd.concat(lst)

    for class_index, group in _dataset.groupby('label'):
        print(f'{class_index}: length: {len(group)}')
    return _dataset

##############################################################################################
train_dataset = resample(train_dataset, ratio = 4)
# test_dataset = resample(test_dataset, ratio = 1)


0: length: 2264
1: length: 2264
2: length: 2264
3: length: 2264
4: length: 2264


In [4]:
# mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
from datasets import Dataset
from transformers import ViTImageProcessor
from transformers import AutoImageProcessor

# model_name_or_path = 'google/vit-base-patch16-224-in21k'
model_name_or_path = 'google/vit-large-patch16-384'
# model_name_or_path = 'google/vit-large-patch32-384'
 
# processor = ViTImageProcessor.from_pretrained(model_name_or_path)
image_processor = AutoImageProcessor.from_pretrained(model_name_or_path)

normalize = T.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)

size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)

print(size)

# def Rotation(img):
#     return T.functional.rotate(img, angle=180)

# class custom_rotation(Callable):
#     def __init__(self, angle=180):
#         self.angle = angle

#     def __call__(self, img):
#         return T.functional.rotate(img, angle=self.angle)

#     def __repr__(self):
#         return "custom rotation"

class MyCustomRotation(torch.nn.Module):
    def forward(self, img):  
        if np.random.rand(1)[0] > 0.5:
            return T.functional.rotate(img, angle=180)
        else:
            return img


_transforms = T.Compose([
    T.CenterCrop(1880),
    T.Resize(size, interpolation=T.InterpolationMode.BICUBIC),
    # T.RandomResizedCrop(size),
    T.ToTensor(),
    normalize])

_transforms_train = T.Compose([
    T.CenterCrop(1880),
    T.RandomHorizontalFlip(p = 0.5),
    T.RandomVerticalFlip(p = 0.5),
    # T.RandomRotation(degrees=(-15, 15)),
    # T.RandomApply(
    MyCustomRotation(), #p=0.5),
    # custom_rotation(angle = 180),  #p=0.5), 
    # T.Resize(size, interpolation=T.InterpolationMode.BICUBIC),
    # T.RandomResizedCrop(size),
    # T.ToTensor(),
    # normalize
])

# transform_plus_rand = T.Compose([
#     T.RandomHorizontalFlip(p = 0.5),
#     # T.RandomVerticalFlip(p = 0.5),
# ])

# transform_basic = T.Compose([
#     T.CenterCrop(1880),
# ])

def load_image(path_image, label, mode):
    # load image
    image = Image.open(path_image)

    if mode == 'train':
        # image = transform_basic(image)
        # image = transform_plus_rand(image)
        image = _transforms_train(image)
        return = processor(image, return_tensors='pt')
                                  
        # return _transforms_train(image.convert("RGB"))
    else:
        # image = transform_basic(image)
        # return _transforms(image.convert("RGB"))
        return processor(image, return_tensors='pt')

        
def func_transform(examples):
    
    # loaded_images = [load_image(path, lb, 'train').convert("RGB") for path, lb in zip(examples['image_path'], examples['label'])]
    # _transforms(img.convert("RGB"))
    # inputs = processor(loaded_images, return_tensors='pt')
    # inputs['label'] = examples['label']

    examples["pixel_values"] = [load_image(path, lb, 'train') 
                                for path, lb in zip(examples['image_path'], examples['label'])]
    del examples["image_path"]
    return examples
    
    # return inputs

def func_transform_test(examples):
    
    # loaded_images = [load_image(path, lb, 'test').convert("RGB") for path, lb in zip(examples['image_path'], examples['label'])]
    # inputs = processor(loaded_images, return_tensors='pt')
    # inputs['label'] = examples['label']

    examples["pixel_values"] = [load_image(path, lb, 'test')
                                for path, lb in zip(examples['image_path'], examples['label'])]
    del examples["image_path"]
    return examples
    
    # return inputs

train_ds = Dataset.from_pandas(train_dataset, preserve_index=False)
test_ds = Dataset.from_pandas(test_dataset, preserve_index=False)

# train_dataset = train_ds.map(func_transform, batched=True, fn_kwargs={"mode": 'train'})
# test_dataset = test_ds.map(func_transform, batched=True, fn_kwargs={"mode": 'test'})

prepared_ds_train = train_ds.with_transform(func_transform)
prepared_ds_test = test_ds.with_transform(func_transform_test)

prepared_ds_train = prepared_ds_train.shuffle(seed=42)
prepared_ds_test = prepared_ds_test.shuffle(seed=42)


(384, 384)


In [5]:
from transformers import ViTImageProcessor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_name_or_path)

def transform(example_batch):
    inputs = processor([x.convert("RGB") for x in example_batch['image']], return_tensors='pt')

    inputs['label'] = example_batch['label']
    return inputs

prepared_ds_train = train_dataset.with_transform(transform)
prepared_ds_test = test_dataset.with_transform(transform)

prepared_ds_train = prepared_ds_train.shuffle(seed=42)
prepared_ds_test = prepared_ds_test.shuffle(seed=42)

# prepared_ds_train = train_dataset.with_transform(func_transform)
# prepared_ds_test = test_dataset.with_transform(func_transform_test)




In [31]:
print(len(prepared_ds_train))
print(len(prepared_ds_test))

11560
7026


In [11]:
# oversampled_dataset = MyDataset(prepared_ds_train, transform = final_transform, oversampling = True)

In [5]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [11]:

# !pip install evaluate

In [6]:
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from sklearn.metrics import f1_score #, kappa
# from sklearn import metrics

import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions_proba, labels = eval_pred

    # print(predictions)
    predictions = np.argmax(predictions_proba, axis=1)
    # print(predictions)
    # print(labels)
    result_accuracy = accuracy.compute(predictions=predictions, references=labels)

    result = {
             'accuracy': np.mean([result_accuracy['accuracy']]),
             'kappa': np.mean([cohen_kappa_score(labels, predictions, weights = "quadratic")]),
             # 'quadratic_kappa': np.mean([kappa(labels, predictions, weights = "quadratic")]),
             'f1': np.mean([f1_score(labels, predictions, average='weighted')]),
             # 'roc_auc': np.mean([roc_auc_score(labels, predictions_proba, multi_class='ovr')])
             }

    cm = confusion_matrix(labels, predictions)
    print(cm)

    # print(cohen_kappa_score(labels, predictions))
    # print(result)

    return result


In [10]:
pred = np.array([0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
                ])
trues = np.array([0, 1, 2, 2, 0, 1, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0
                 ])
cohen_kappa_score(trues, pred, weights = "quadratic")

cm = confusion_matrix(trues, pred)
print(cm)
    

[[29  0  0]
 [ 1  1  0]
 [ 5  0  1]]


In [7]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    ignore_mismatched_sizes=True,
    num_labels=5
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-large-patch16-384 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([5, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# !pip install accelerate -U
# 

In [15]:
# !pip install transformers -U

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./vit-base",
    
    report_to="wandb",  # enable logging to W&B
    run_name="vit384(16)_rot_4",  # name of the W&B run (optional)
    logging_steps=20,  # how often to log to W&B
    per_device_train_batch_size=8,
    evaluation_strategy="steps",
    num_train_epochs=2,
    fp16=True,
    save_steps=20,
    eval_steps=20,
    # label_smoothing_factor = 0.96,
    learning_rate=5e-5,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    load_best_model_at_end=True,
    metric_for_best_model="kappa", 
    greater_is_better = True
)

In [9]:
sample_ids = np.random.choice(len(prepared_ds_test), size=200, replace=False)
sample_ids_train = np.random.choice(len(prepared_ds_train), size=len(prepared_ds_train), replace=False)
val_ds = prepared_ds_test.select(sample_ids)
train_ds_shiffled = prepared_ds_train.select(sample_ids_train)

In [10]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds_train,
    # eval_dataset=prepared_ds_test,
    eval_dataset=val_ds,
    tokenizer=image_processor,
)

In [11]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

wandb: Currently logged in as: kalexu97 (alexu97skoltech). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,Kappa,F1
20,1.637700,1.546638,0.165000,0.174422,0.144416
40,1.320400,1.815674,0.165000,0.163120,0.119453
60,1.398000,1.640361,0.245000,0.183101,0.302324
80,1.380100,1.564199,0.250000,0.251880,0.303070
100,1.248300,1.160047,0.610000,0.307177,0.642663
120,1.416300,1.074678,0.615000,0.280198,0.609886
140,1.309100,1.208398,0.505000,0.398575,0.565160
160,1.290600,1.558871,0.170000,0.297781,0.156017
180,1.265500,1.229059,0.335000,0.204842,0.375223
200,1.455300,1.965866,0.115000,0.230576,0.053613


[[ 12 106  14   7   7]
 [  0  16   1   0   0]
 [  0  22   1   3   3]
 [  0   2   1   3   0]
 [  0   0   0   1   1]]
[[ 6 56 79  3  2]
 [ 0  5 11  1  0]
 [ 2  6 16  4  1]
 [ 0  0  1  5  0]
 [ 0  0  0  1  1]]
[[36 69  0 21 20]
 [ 2  7  0  4  4]
 [ 1 13  0 10  5]
 [ 0  0  0  5  1]
 [ 0  0  0  1  1]]
[[36 80  2  8 20]
 [ 1 11  0  4  1]
 [ 2 14  0  4  9]
 [ 0  0  0  1  5]
 [ 0  0  0  0  2]]
[[107   3  18   3  15]
 [  6   2   7   1   1]
 [ 11   2   9   1   6]
 [  1   0   0   2   3]
 [  0   0   0   0   2]]
[[113  21  12   0   0]
 [ 10   5   2   0   0]
 [ 22   1   4   2   0]
 [  3   0   3   0   0]
 [  0   0   1   0   1]]
[[83 48  6  3  6]
 [ 6  8  1  1  1]
 [ 8  8  5  2  6]
 [ 0  1  1  4  0]
 [ 0  0  0  1  1]]
[[  7 134   4   0   1]
 [  0  15   2   0   0]
 [  0  20   8   1   0]
 [  0   2   1   3   0]
 [  0   1   0   0   1]]
[[ 43   0 103   0   0]
 [  2   0  14   1   0]
 [  4   0  19   3   3]
 [  0   0   2   4   0]
 [  0   0   1   0   1]]
[[  0 110  22   2  12]
 [  0  13   3   1   0]
 [  0  15 

In [12]:
metrics = trainer.evaluate(prepared_ds_test)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

[[3184 1325  600    3   40]
 [ 191  184   87    1    1]
 [ 191  194  567   80   70]
 [   8    9   49   73   27]
 [   6    1   27   22   86]]
***** eval metrics *****
  epoch                   =        2.0
  eval_accuracy           =     0.5827
  eval_f1                 =     0.6386
  eval_kappa              =     0.5631
  eval_loss               =     0.9655
  eval_runtime            = 0:13:37.82
  eval_samples_per_second =      8.591
  eval_steps_per_second   =      1.075
